In [7]:
import random

def hangman(secret_word, guesser, max_mistakes=8, verbose=True, **guesser_args):
    validLetters = 'abcdefghijklmnopqrstuvwxyz'
    secret_word = secret_word.lower()
    mask = ['_'] * len(secret_word)
    guessed = set()
    
    print("『Hangman』，hints：", ' '.join(mask), 'character : ', len(secret_word))
    mistakes = 0
    
    while mistakes < max_mistakes:
        #print(max_mistakes-mistakes, "turns left")
        guess = guesser(mask, guessed, **guesser_args)
        
        if guess in guessed:
            print('Try Again！,used')
            mistakes += 1
        elif guess not in validLetters:
            print("Try Again！,Not valid character")
            mistakes += 1
        else:
            guessed.add(guess)
            if guess in secret_word:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
                print('Guess:，', ' '.join(mask))
            else: 
                mistakes += 1

        if '_' not in mask:
            return mistakes
        
    print('『You loose』，The Ans :', secret_word) 
    print("You loose, You let a kind man die")
    print("  --------  ")
    print("     O_|    ")
    print("    /|\      ")
    print("    / \     ")
    return mistakes

def human(mask, guessed, **kwargs):
    return input().lower().strip()

# word = random.choice(["ant", "pugger" , "tiger" , "pokemon" , "water" , "earth"])
# hangman(word, human, 5, True)

## 準備訓練和測試集

開始寫我們的AI猜字之前，要先準備訓練集和測試集。我們使用之前用過的Brown Corpus來訓練AI猜字演算法，為了刻意增加猜字的難度，AI在測試時的字和訓練的字會是不一樣的(把訓練集和測試集拆開)，如此才能訓練出更通用的模型。

我們使用 nltk.corpus.Brown 裡的 words() 方法，從中將非英文字母的文字去除，同時將每個字小寫化。之後，我們用 numpy.random.shuffle 來隨機刷取文集中的字，以分成訓練集和測試集。測試集裡有1000字，剩下的字都會到訓練集裡面。

In [2]:
import nltk
nltk.download('brown')

from nltk.corpus import brown
import numpy as np

np.random.seed(12345)
word_set = []      # word_set 儲存Brown Corpus中所有獨特的字型
test_set = []      # test_set 儲存1000個字的測試集
training_set = []  # training_set 將剩下的字存到訓練集中
word_s = set([])

for word in brown.words():
    if word.isalpha():
        word = word.lower()
        if word not in word_s:
            word_s.add(word)

word_set = list(word_s)
np.random.shuffle(word_set)

test_set = word_set[:1000]
training_set = word_set[1000:]

print(len(word_set))
print(len(test_set))
print(len(training_set))

hangman(np.random.choice(test_set), human, 5 , True)

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\cti110016\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


40234
1000
39234
『Hangman』，hints： _ _ _ _ character :  4


 a
 b
 c


Guess:， _ _ c _


 d


...

 e


Guess:， d _ c e


 f
 g
 h


『You loose』，The Ans : dice
You loose, You let a kind man die
  --------  
     O_|    
    /|\      
    / \     


5

## AI猜字方法：參考 rainrush
- 1.Random

In [3]:
import string

def test_guesser(guesser, test=test_set):
    """
        這個方法是用來測試平均猜錯的次數。
    """
    total = 0
    for word in test:
        total += hangman(word, guesser, 26, False)
    return total / float(len(test))

def random_guesser(mask, guessed, **kwargs):
    alphabets = []
    for letter in range(97,123):
        if chr(letter) not in guessed:
            alphabets.append(chr(letter))

    picked = np.random.choice(alphabets)
    return picked

# 若要看看機器是怎麼猜字的，可以把下面這句打開
#hangman(np.random.choice(test_set), random_guesser, 10, True)

result = test_guesser(random_guesser)
print("Average number of incorrect guesses: ", result)

『Hangman』，hints： _ _ _ _ _ _ _ _ _ _ _ character :  11
Guess:， _ _ _ _ _ r _ _ _ _ _
Guess:， _ _ l _ _ r _ _ _ _ _
Guess:， c _ l _ _ r _ _ _ _ _
Guess:， c _ l _ b r _ _ _ _ _
Guess:， c _ l i b r _ _ i _ _
Guess:， c _ l i b r _ _ i _ n
Guess:， c _ l i b r _ t i _ n
Guess:， c _ l i b r _ t i o n
Guess:， c a l i b r a t i o n
『Hangman』，hints： _ _ _ _ _ _ _ _ character :  8
Guess:， _ _ _ _ _ _ n _
Guess:， _ _ r _ _ _ n _
Guess:， c _ r _ _ _ n _
Guess:， c _ r _ _ _ n s
Guess:， c _ r t _ _ n s
Guess:， c _ r t o o n s
Guess:， c a r t o o n s
『Hangman』，hints： _ _ _ _ _ character :  5
Guess:， _ _ _ i _
Guess:， c _ c i _
Guess:， c _ c i l
Guess:， c e c i l
『Hangman』，hints： _ _ _ _ _ _ _ _ _ _ character :  10
Guess:， _ _ _ g _ _ _ _ _ g
Guess:， f _ _ g _ _ _ _ _ g
Guess:， f _ _ g _ t t _ _ g
Guess:， f _ _ g e t t _ _ g
Guess:， f _ _ g e t t i _ g
Guess:， f _ r g e t t i _ g
Guess:， f o r g e t t i _ g
Guess:， f o r g e t t i n g
『Hangman』，hints： _ _ _ _ _ _ _ _ _ _ character :  10
Guess:， _ _ _ _

- 第二種猜法：Unigram Guesser

我們可以嘗試用Unigram模型來訓練。我們需要知道每個字母的出現頻率，接著照出現頻率的高低來進行猜字。每當猜完一個字之後就應該把猜過的字去掉。

In [4]:
from collections import Counter 
unigram_counts = Counter() # 儲存了整個訓練及中每個字母的出現次數

for word in training_set:
    for letter in word:
        unigram_counts[letter] += 1

print(unigram_counts)


def unigram_guesser(mask, guessed, unigram_counts=unigram_counts):
    unigram_keys = []

    # 照出現頻率將字母排序
    for i in range(len(unigram_counts)):
        unigram_keys.append(unigram_counts.most_common()[i][0])

    # 將猜過的字去除
    for letter in guessed:
        if letter in unigram_keys:
            unigram_keys.remove(letter)

    return unigram_keys[0]

#hangman(np.random.choice(test_set), unigram_guesser, 10, True)
result = test_guesser(unigram_guesser)
print("平均猜錯次數：", result)

Counter({'e': 35220, 'i': 26080, 'a': 24353, 's': 23733, 'n': 22635, 'r': 22114, 't': 20646, 'o': 19171, 'l': 16670, 'c': 12643, 'd': 12257, 'u': 10186, 'm': 8653, 'g': 8634, 'p': 8540, 'h': 7328, 'b': 5743, 'y': 5350, 'f': 4226, 'v': 3436, 'k': 2973, 'w': 2794, 'z': 1030, 'x': 835, 'j': 634, 'q': 529})
『Hangman』，hints： _ _ _ _ _ _ _ _ _ _ _ character :  11
Guess:， _ _ _ i _ _ _ _ i _ _
Guess:， _ a _ i _ _ a _ i _ _
Guess:， _ a _ i _ _ a _ i _ n
Guess:， _ a _ i _ r a _ i _ n
Guess:， _ a _ i _ r a t i _ n
Guess:， _ a _ i _ r a t i o n
Guess:， _ a l i _ r a t i o n
Guess:， c a l i _ r a t i o n
Guess:， c a l i b r a t i o n
『Hangman』，hints： _ _ _ _ _ _ _ _ character :  8
Guess:， _ a _ _ _ _ _ _
Guess:， _ a _ _ _ _ _ s
Guess:， _ a _ _ _ _ n s
Guess:， _ a r _ _ _ n s
Guess:， _ a r t _ _ n s
Guess:， _ a r t o o n s
Guess:， c a r t o o n s
『Hangman』，hints： _ _ _ _ _ character :  5
Guess:， _ e _ _ _
Guess:， _ e _ i _
Guess:， _ e _ i l
Guess:， c e c i l
『Hangman』，hints： _ _ _ _ _ _ _ _ _ _ cha

- 第三種猜法：根據文字長度猜字: 不同的文字長度，每個字母出現的頻率不盡相同，例如，短的字比較不會出現前綴或後綴。在這裡，我們針對不同的文字長度設計不一樣的猜字順序

In [5]:
from collections import defaultdict

# unigram_counts_by_length 將文字長度和字母頻率map在一起
unigram_counts_by_length = defaultdict(Counter)

# 幫每一種文字長度寫不同的Unigram Model
for word in training_set:    
    this_count = Counter()
    for letter in word:
        this_count[letter] += 1
        unigram_counts_by_length[len(word)] += this_count
        this_count = Counter()

def exclude_guessed_letters(length_model, guessed):
    unigram_keys_by_length = []
    # 照出現頻率將字母排序
    for i in range(len(unigram_counts_by_length[length_model])):
        unigram_keys_by_length.append(unigram_counts_by_length[length_model].most_common()[i][0])
    
    # 將猜過的字去除
    for letter in guessed:
        if letter in unigram_keys_by_length:
            unigram_keys_by_length.remove(letter)
    return unigram_keys_by_length


lengths = sorted(unigram_counts_by_length.keys())
max_length = lengths[-1] + 1
#print(unigram_counts_by_length)

def unigram_length_guesser(mask, guessed, counts=unigram_counts_by_length):
    
    length_model = len(mask)
    # 若要猜的文字長度不在unigram model時，我們將一長度來猜。
    while length_model not in lengths:
        length_model -= 1
    
    unigram_keys_by_length = exclude_guessed_letters(length_model, guessed)
    
    # 若這個文字長度沒有猜字選項了，從附近的文字長度找
    while len(unigram_keys_by_length) == 0:
        if length_model < 20:
            length_model += 1
        else:
            length_model -= 1
        unigram_keys_by_length = exclude_guessed_letters(length_model, guessed)
    
    return unigram_keys_by_length[0]
#hangman(np.random.choice(test_set), unigram_length_guesser, 10, True)
result = test_guesser(unigram_length_guesser)
print("平均猜錯次數：", result)

『Hangman』，hints： _ _ _ _ _ _ _ _ _ _ _ character :  11
Guess:， _ _ _ i _ _ _ _ i _ _
Guess:， _ _ _ i _ _ _ _ i _ n
Guess:， _ _ _ i _ _ _ t i _ n
Guess:， _ a _ i _ _ a t i _ n
Guess:， _ a _ i _ r a t i _ n
Guess:， _ a _ i _ r a t i o n
Guess:， _ a l i _ r a t i o n
Guess:， c a l i _ r a t i o n
Guess:， c a l i b r a t i o n
『Hangman』，hints： _ _ _ _ _ _ _ _ character :  8
Guess:， _ _ _ _ _ _ _ s
Guess:， _ a _ _ _ _ _ s
Guess:， _ a _ _ _ _ n s
Guess:， _ a r _ _ _ n s
Guess:， _ a r t _ _ n s
Guess:， _ a r t o o n s
Guess:， c a r t o o n s
『Hangman』，hints： _ _ _ _ _ character :  5
Guess:， _ e _ _ _
Guess:， _ e _ _ l
Guess:， _ e _ i l
Guess:， c e c i l
『Hangman』，hints： _ _ _ _ _ _ _ _ _ _ character :  10
Guess:， _ _ _ _ e _ _ _ _ _
Guess:， _ _ _ _ e _ _ i _ _
Guess:， _ _ _ _ e _ _ i n _
Guess:， _ _ _ _ e t t i n _
Guess:， _ _ r _ e t t i n _
Guess:， _ o r _ e t t i n _
Guess:， _ o r g e t t i n g
Guess:， f o r g e t t i n g
『Hangman』，hints： _ _ _ _ _ _ _ _ _ _ character :  10
Guess:， _ i _ _

- 強化版本

除了考慮字母出現的機率和在各種長度中各個字母出現的機率，今天我們也把字母的排列順序列入考量。例如，我們看到一個字 m _ s s，我們知道他有很高的機率會是母音，因此，a, e, i, o, u就是我們下一個猜測的目標。

在這裡，我們開發一個Bigram的語言模型。在開始訓練這個模型之前，有幾件事需要注意：(1) 要記得加上開頭標籤 $；(2) 要記得針對Bigram以上的模型設計對應的平滑方法。在這裡我們決定使用 linear interpolation （線性插值法）來smooth高元和低元模型，而插值法中要用到的 \lambda 值我們設為0.75（建議在0.7~0.8之間）。

我們猜格子的順序會從最左邊開始猜，因此最開始會在p($)的條件下猜字母。猜的過程中可能第一個字還沒猜到，後面有些就先被猜到了，等我們終於猜到第一個字母後，我們就找下一個「最左邊還沒有被猜到的字」。

In [6]:
from operator import itemgetter
bigram_counts = defaultdict(Counter) # 這個dict的key是各個字母，value是可能接在這個字母後面之字母的Counter
bigram_inter = defaultdict(list) # 根據interpolation後的機率，將接續在一個字母後面的可能字母照出現機率排序

# 開始儲存Bigram model
for word in training_set:
    this_count = Counter()
    word = '$' + word # 字首加上開頭標籤
    for i, letter in enumerate(word):
        if i+1 != len(word):
            this_count[word[i+1]] += 1
            bigram_counts[letter] += this_count
            this_count = Counter()

set_lambda = 0.75 # 設定interpolation的lambda

# Bigram Interpolated Model: p = lambda*p(w_i|w_(i-1)) + (1-lambda)*p(w_i)
# p(w_i|w_(i-1)) = count(w_i and w_(i-1)) / count(w_(i-1))
# p(w_i) = count(w_i) / sigma(count(w_i))
for key in bigram_counts.keys():
    sigma_count_wi = sum(unigram_counts.values()) # sigma(count(w_i))
    count_wi1 = sum(bigram_counts[key].values()) # count(w_(i-1))

    # 計算p(w_i|w_(i-1))和p(w_i)
    prob_of_letter = {}
    for letter in range(97,123):
        p_wi_wi1 = bigram_counts[key][chr(letter)] / count_wi1 # p(w_i|w_(i-1))
        p_wi = unigram_counts[chr(letter)] / sigma_count_wi # p(w_i)
        prob_of_letter[chr(letter)] = set_lambda*p_wi_wi1 + (1-set_lambda)*p_wi

    # 將接續在一個字母後面的可能字母照出現機率排序
    this_list = []
    for i in range(26):
        this_list.append(sorted(prob_of_letter.items(), key=itemgetter(1), reverse=True)[i][0])
    bigram_inter[key] = this_list
    

def bigram_guesser(mask, guessed, counts=bigram_counts): # add extra default arguments if needed
    mask = ['$'] + mask
    w_i_1 = "" # 找到最左邊非 '_' 的字母 -> w_(i-1)
    # 找w_i_1
    for i in range(len(mask)):
        if mask[i] == '_':
            w_i_1 = mask[i-1]
            break
    copy_bi_model = bigram_inter[w_i_1].copy()
    
    # 將已經猜過的字母去除
    for letter in guessed:
        if letter in copy_bi_model:
            copy_bi_model.remove(letter)

    return copy_bi_model[0]


#hangman(np.random.choice(test_set), bigram_guesser, 26, True)

result = test_guesser(bigram_guesser)
print("平均猜錯次數：", result)

『Hangman』，hints： _ _ _ _ _ _ _ _ _ _ _ character :  11
Guess:， c _ _ _ _ _ _ _ _ _ _
Guess:， c _ _ _ _ _ _ _ _ o _
Guess:， c a _ _ _ _ a _ _ o _
Guess:， c a _ _ _ _ a t _ o _
Guess:， c a _ _ _ _ a t _ o n
Guess:， c a _ _ _ r a t _ o n
Guess:， c a l _ _ r a t _ o n
Guess:， c a l i _ r a t i o n
Guess:， c a l i b r a t i o n
『Hangman』，hints： _ _ _ _ _ _ _ _ character :  8
Guess:， _ _ _ _ _ _ _ s
Guess:， c _ _ _ _ _ _ s
Guess:， c _ _ _ o o _ s
Guess:， c a _ _ o o _ s
Guess:， c a _ t o o _ s
Guess:， c a _ t o o n s
Guess:， c a r t o o n s
『Hangman』，hints： _ _ _ _ _ character :  5
Guess:， c _ c _ _
Guess:， c e c _ _
Guess:， c e c i _
Guess:， c e c i l
『Hangman』，hints： _ _ _ _ _ _ _ _ _ _ character :  10
Guess:， _ _ _ _ e _ _ _ _ _
Guess:， _ _ r _ e _ _ _ _ _
Guess:， _ _ r _ e t t _ _ _
Guess:， _ _ r _ e t t i _ _
Guess:， f _ r _ e t t i _ _
Guess:， f o r _ e t t i _ _
Guess:， f o r _ e t t i n _
Guess:， f o r g e t t i n g
『Hangman』，hints： _ _ _ _ _ _ _ _ _ _ character :  10
Guess:， _ _ s _